In [1]:
import math
import numpy as np
import math
# import pandas
from optparse import OptionParser
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble
from collections import defaultdict
from copy import deepcopy
from multiprocessing import Pool
from itertools import chain
import time
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from __future__ import division

In [2]:
# Functions to extrac the documents, query and rank information
def extractFeatures(split):
    features = []
    for i in range(2, 138):
        features.append(float(split[i].split(':')[1]))
    # Convert to tuples:
    return features

def extractQueryData(split):
    # Add tuples:
    queryFeatures = split[1].split(':')[1]
    return queryFeatures

def readDataset(path):
    dictio_quid= defaultdict(list)
    features_list=[]
    rank_list=[]
    print('Reading training data from file...')
    k=0
    with open(path, 'r') as file:
        for line in file:
            # Getting features:
            split = line.split()
            features=extractFeatures(split)
            features_list.append(features)

            # Getting the query:
            query = int(extractQueryData(split))
            #print "query: "+str(query)
            # Getting rank:
            rank = int(split[0])
            rank_list.append(rank)

            # Feeding dictionary:
            dictio_quid[query].append((features, rank))
            k+=1
            #if k==10:
            #   break
    print('Number of query ID %d' %(len(features_list)))
    return np.array(features_list), np.array(rank_list), dictio_quid

# Normalisation:
def normalize_features(features):
    features=np.array(features)

    # Substracting the mean:
    mean_features = np.mean(features, axis=0)
    features = features - mean_features

    # Dividing by the std:
    std_features = np.std(features, axis=0)
    features = features / std_features
    #print "features normalized"
    return features, mean_features, std_features

In [3]:
# ORDERING for evaluation:
def evaluate(model, dictio_val, mean_Xval, std_Xval):
    dictio_evaluation = defaultdict(list)

    for key in dictio_val.keys():
        temp_list = dictio_val[key]
        for features_vec, relevance in temp_list:
            # Features:
            features_norm = (np.array(features_vec) - mean_Xval) / std_Xval
            features_norm = features_norm.reshape(1,-1)
            #features_norm = features_norm.reshape(-1,1)

            # Prediction:
            prediction = model.predict(features_norm)

            # Dictionary:
            dictio_evaluation[key].append((prediction[0], relevance))

            # print features_vec,relevance

    return dictio_evaluation
def reorder_dictio(dictio_eval):
    for key in dictio_eval.keys():
        #dictio_eval[key]=sorted(dictio_eval[key], reverse=True,key=lambda tup: (tup[1], tup[0]))
        dictio_eval[key] = sorted(dictio_eval[key], reverse=True, key=lambda tup: tup[1])
    return dictio_eval

def txt_to_dictio_eval(file_name):
    
    model_output_array = np.loadtxt(file_name)
    model_output_arr_rel = np.concatenate((model_output_array, np.reshape(tes_ranks, (-1, 1))), axis=1)
    
    model_output_dict = defaultdict(list)
    model_output_dict_or = defaultdict(list)

    for i in model_output_arr_rel:
        model_output_dict[i[0]].append((i[3], i[2]))

    for i in model_output_dict.keys():
        model_output_dict[i].sort(reverse=True)

    for i in model_output_dict.keys():
        model_output_dict_or[i] = [(x[1] ,x[0]) for x in model_output_dict[i]]
        
    return model_output_dict_or

In [4]:
# NDCG:
def dcg_score(y_true, y_score, k=10, gains="exponential"):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def ndcg_score(y_true, y_score, k=10, gains="exponential"):
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_score, k, gains)
    return actual / best

def separate(relevance_tuple):
    y_true=[]
    y_pred = []
    for tup in relevance_tuple:
        y_pred.append(tup[0])
        y_true.append(tup[1])
    return y_true,y_pred

def ndcg(dictio_eval):
    """
    """
    b= reorder_dictio(dictio_eval)
    new_b=[]
    for qid in b.keys():
        r_true=[]
        r_pred=[]
        for i,j in zip(pd.DataFrame(b[qid])[0], pd.DataFrame(b[qid])[1]):
            r_pred.append(i)
            r_true.append(j)
        score=ndcg_score(r_true,r_pred)    
        #new_b[qid]= [r_true, r_pred]
        new_b.append(score)
    return new_b

In [5]:
## ERR:

def get_bins(dictio_eval):
    #Get the bins:
    all_tuples=[]
    for key in dictio_eval.keys():
        all_tuples.append(dictio_eval[key])
    all_tuples=list(chain.from_iterable(all_tuples))
    all_scores=[i[0] for i in all_tuples]
    all_true=[i[1] for i in all_tuples]
    number_bins=4#len(set(all_true))
    bins = np.histogram(all_scores, bins=number_bins, range=None, normed=False, weights=None)[1]
    return bins

def assign_to_bin(list_score,bins):
    # Get bins
    #bins = np.histogram(list_score, bins=number_bins, range=None, normed=False, weights=None)[1]
    inds = np.digitize(list_score, bins,right=True)
    return inds

# Functions to calculate ERR:

GAMMA=0.450
def get_proba2(list_tuples,bins):
    list_proba=[]
    list_score = [i[0] for i in list_tuples]
    list_true = [i[1] for i in list_tuples]
    pred_relevance = assign_to_bin(list_score,bins)
    list_tuples=zip(pred_relevance,list_true)
    for r_pred,r_true in list_tuples:
        proba = ((np.power(2,r_pred))-1)/ np.power(2,4)#np.max(r_pred))
        list_proba.append(proba)
    return list_proba

## Different:
def get_proba(list_tuples,bins):
    list_proba=[]
    list_score = [i[0] for i in list_tuples]
#    print list_score
    list_true = range(len(list_tuples))
#   print list_true
    pred_relevance = assign_to_bin(list_score,bins)
#    print pred_relevance
    list_tuples=zip(pred_relevance,list_true)
    for r_pred,r_true in list_tuples:
        proba = ((np.power(2,r_pred))-1)/ np.power(2,4)#np.max(r_pred))
        list_proba.append(proba)
    return list_proba

def get_ERR(list_proba,n=10,gamma=0.5):
    r=2
    err = list_proba[0]
    last_proba=1
    for i in xrange(1,len(list_proba)):
        actual_proba=list_proba[i]
        previous_proba=(1-list_proba[i-1])*last_proba
        #print proba
        stop_proba=actual_proba*previous_proba
        err+=stop_proba/r
        last_proba=previous_proba
        r+=1
    return err

def ERR(dictio_eval,n=10,gamma=GAMMA):
    list_ERR=[]
    # Get the bins:
    bins=get_bins(dictio_eval)
    for key in dictio_eval.keys():
        list_tuples=dictio_eval[key]
        list_proba=get_proba(list_tuples,bins)
        err_result=get_ERR(list_proba,n,gamma)
        list_ERR.append(err_result)
    return list_ERR

In [ ]:
## MAIN:

In [7]:
tes_features, tes_ranks, tes_queries= readDataset('./MSLR-WEB10K/Fold1/test.txt')
tes_features, mean_Xval, std_Xval= normalize_features(tes_features)

Reading training data from file...
Number of query ID 241521


In [8]:
# Getting the list of scores:
import os
list_scores=[]
for file in os.listdir("./best_models/"):
    if file.endswith(".txt"):
        list_scores.append(os.path.join("./best_models/", file))

In [9]:
list_scores

['./best_models/test_LM_rank_score_LambdaMART1.txt',
 './best_models/test_RB_rank_score_RB_model_r100_tc10.txt',
 './best_models/test_RN_rank_score_RN_model_e20_l2_n1_lr5e-05.txt']

In [10]:
model_names=["LambdaMart","RankBoost","RankNet"]

In [11]:
# Extract
dict_scores=defaultdict(list)
gammas=[0.65,0.35,0.25]
i=0
for best_score,key in zip(list_scores,model_names):
    dictio_eval = txt_to_dictio_eval(best_score)
    lst_ndcg = ndcg(dictio_eval)
    lst_ERR = ERR(dictio_eval,gamma=gammas[i])
    print (np.nanmean(lst_ndcg), np.nanmean(lst_ERR))
    dict_scores[key].append(np.nanmean(lst_ndcg))
    dict_scores[key].append(np.nanmean(lst_ERR))
    i+=1

/Users/Chelo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars


(0.46181678515272756, 0.47330885960864894)
(0.37704545135360484, 0.50873371130849543)
(0.23251493220839448, 0.40672594862600914)


In [35]:
query=58
dict_query58=defaultdict(list)
for best_score,key in zip(list_scores,model_names):
    dictio_eval = txt_to_dictio_eval(best_score)
    list_tuples = dictio_eval[query][0:10]
    dict_query58[key].append(list_tuples)

In [40]:
for i in dict_query58["RankNet"][0]:
    print i

(0.39973296644484663, 4.0)
(0.39973296644484663, 4.0)
(0.39973296644484663, 3.0)
(0.39973296644484663, 3.0)
(0.42531100212049794, 2.0)
(0.39973296644484663, 2.0)
(0.39973296644484663, 1.0)
(0.39973296644484663, 1.0)
(0.39973296644484663, 1.0)
(0.39973296644484663, 1.0)


In [27]:
query=58
rank_dict=defaultdict(list)
for score,key in zip(list_scores,model_names):
    dictio_eval = txt_to_dictio_eval(score)
    # Get first 10 results:
    list_tuples = dictio_eval[query][0:10]
    list_score = [i[0] for i in list_tuples]
    list_true = [i[1] for i in list_tuples]
    bins=get_bins(dictio_eval)
    # Get predicted relevance:
    pred_relevance = assign_to_bin(list_score,bins)
    list_tuples=zip(pred_relevance,list_true)
    rank_dict[key].append(list_tuples)

In [33]:
rank_dict

defaultdict(list,
            {'LambdaMart': [[(3, 4.0),
               (2, 4.0),
               (3, 3.0),
               (2, 3.0),
               (2, 2.0),
               (2, 2.0),
               (2, 1.0),
               (2, 1.0),
               (2, 1.0),
               (2, 1.0)]],
             'RankBoost': [[(3, 4.0),
               (1, 4.0),
               (2, 3.0),
               (1, 3.0),
               (2, 2.0),
               (1, 2.0),
               (2, 1.0),
               (2, 1.0),
               (2, 1.0),
               (2, 1.0)]],
             'RankNet': [[(0, 4.0),
               (0, 4.0),
               (0, 3.0),
               (0, 3.0),
               (4, 2.0),
               (0, 2.0),
               (0, 1.0),
               (0, 1.0),
               (0, 1.0),
               (0, 1.0)]]})

In [18]:
table_best=pd.DataFrame.from_dict(dict_scores,"index")
table_best.columns=["NDCG@10","ERR@10"]
table_best.to_csv("final_results.csv")